In [ ]:
# default_exp utils

# utils

> Utility functions for the _point_to_box_ package.

In [ ]:
#hide
%reload_ext autoreload
%autoreload 2
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#hide
#export
import cv2
from cv2 import rectangle
import numpy as np

In [ ]:
#export
def get_norm_stats(loaders) -> list:
    """Returns normalization stats (mean and std) computed across one or more image dataloaders.
     
    **Parameters**
    
    loaders : list of pytorch dataloaders
        
    **Returns**
        
    list of stats as pytorch tensors
    
    """
    mean = 0.
    std = 0.
    nb_samples = 0.
    for loader in loaders:
        for data, _ in loader:
            batch_samples = data.size(0)
            data = data.view(batch_samples, data.size(1), -1)
            mean += data.mean(2).sum(0)
            std += data.std(2).sum(0)
            nb_samples += batch_samples

    mean /= nb_samples
    std /= nb_samples

    return [mean, std]

In [ ]:
# TO-DO: write test

In [ ]:
#export
def draw_rect(im, cords, box_format = None, color = None):
    """
    Draw one or more rectangls on an image.
    
    **Parameters**
    
    im : numpy image [W, H, C]
    
    cords : Numpy array containing N bounding boxes [N, 4]
    
    box_format : String indicating bbox-coordinate format
        
     - coco     : [xmin, ymin, w, h]
     - cnt_ofst : [xofst, yofst, w, h]
     - *_frac   : as fraction of image width/height
     - None     : [xmin, ymin, xmax, ymax]
        
        
    color : List of one or more colors [(R, G, B)]
        
    **Returns**
    
    numpy image with bounding boxes drawn on it [W, H, C]
        
    """
    if color:
        assert len(color) in [1 or len(cords)], 'Improper color specification'
        if len(color) == 1:
            color *= len(cords)
    else:
        color = [(0,255,0)]*len(cords)
    
    im = im.copy()
    
    cords = cords[:,:4]
    cords = cords.reshape(-1,4)
        
    for i, cord in enumerate(cords):
        pt1, pt2 = [],[]
        if box_format == 'coco':
            # cord format: [xmin, ymin, boxw, boxh]
            pt1 = (cord[0], cord[1])
            pt2 = (cord[0] + cord[2], cord[1] + cord[3])
            
        elif box_format == 'cntr_ofst_frac':
            # cord format: [xofst_frac, yofst_frac, frac_boxw, frac_boxh]
            imgh, imgw = im.shape[:-1]
            img_cntr_x, img_cntr_y = imgw/2, imgh/2
            boxw = cord[2] * imgw
            boxh = cord[3] * imgh
            x_ofst = cord[0]*(imgw/2)
            y_ofst = cord[1]*(imgh/2)
            xmin = img_cntr_x + x_ofst - boxw/2
            ymin = img_cntr_y + y_ofst - boxh/2
            pt1 = (xmin, ymin)
            pt2 = (xmin + boxw, ymin + boxh)

        elif box_format == 'corner_ofst_frac':
            # cord format: [xofst_frac, yofst_frac, boxw_farc, boxh_farc]
            imgh, imgw = im.shape[:-1]
            w, h = cord[2] * imgw, cord[3] * imgh
            xmin, ymin = cord[0]*imgw - w/2, cord[1]*imgh - h/2
            pt1 = (xmin , ymin)
            pt2 = (xmin + w, ymin + h)
            
        elif box_format == 'cntr_ofst':
            # cord format: [xofst, yofst, boxw, boxh]
            imgh, imgw = im.shape[:-1]
            img_cntr_x, img_cntr_y = imgw/2, imgh/2
            xmin = img_cntr_x + cord[0] - cord[2]/2
            ymin = img_cntr_y + cord[1] - cord[3]/2
            pt1 = (xmin, ymin)
            pt2 = (xmin + cord[2], ymin + cord[3])
            
        else:
            # cord format: [xmin, ymin, xmax, ymax]
            pt1, pt2 = (cord[0], cord[1]) , (cord[2], cord[3])
                
        pt1 = int(pt1[0]), int(pt1[1])
        pt2 = int(pt2[0]), int(pt2[1])
    
        im = rectangle(im.copy(), pt1, pt2, color[i], int(max(im.shape[:2])/200))
    return im

In [ ]:
# TO-DO: write test for convert cords

In [ ]:
#export
def convert_cords(cords, img_dims, cord_format):
    """
    Convert bounding box coordinates
    
    **Parameters**
    
    cords : list of bbox coordinates [xmin, ymin, w, h]
    
    img_dims : Image dimensions as list or tuple
        
    cord_format : Coordinate conversion format
    
    - cnt_ofst         : [xofst, yofst, w, h]
    - cntr_ofst_frac   : [xofst, yofst, w, h] as fraction of image width/height
    - corner_ofst_frac : [xmin, min, w, h] as fraction of image width/height
        
    **Returns**
    
    List of converted box coordinates
    
    """
    box_cntr = (xmin + (w/2), ymin + (h/2))
    img_cntr = (imgw/2, imgh/2)
    img_dims = (imgw, imgh)

    ofst = []
    for bx_cord, img_cord, img_dim in zip(box_cntr, img_cntr, img_dims):
        if cord_format == 'cntr_ofst':
            ofst.append(bx_cord - img_cord)
        elif cord_format == 'cntr_ofst_frac':
            ofst.append((bx_cord - img_cord)/(img_dim/2))
        elif cord_format == 'corner_ofst_frac':
            ofst.append(bx_cord/img_dim)

    if cord_format in ['cntr_ofst_frac',
                       'corner_ofst_frac']:
        w, h = w/imgw, h/imgh

    return [ofst[0], ofst[1], w, h]

In [ ]:
#export
def resize(size, img, bbox):
    """
    Resize an image in accordance to 'image_letter_box' function in darknet
    
    **Params**
    
    size : target size for the long-edge of the original image
    
    img : np.ndarray image to be resized, shape HxWxC
    
    bbox : np.ndarray of bounding box coordinates [xmin, ymin, xmax, ymax]
    
    **Returns**
    
    img : resized image as np.ndarray
    
    bbox : resized bounding box np.ndarray [xmin, ymin, xmax, ymax]
    
    """
    
    img_w, img_h = img.shape[1], img.shape[0]
#     print(img.shape)
    w, h = size, size
    new_w = int(img_w * min(w/img_w, h/img_h))
    new_h = int(img_h * min(w/img_w, h/img_h))
#     new_w = size
#     new_h = size
    resized_image = cv2.resize(img, (new_w,new_h))
#     print(f'Resized shape: {resized_image.shape}')
#     print(f'size: {size}')
#     print(f'new_w: {new_w}')
#     print(f'new_h: {new_h}')
#     print(f'Image shape: {resized_image.shape}')
    canvas = np.full((size, size, 3), 0)
#     print(f'Canvas shape: {canvas.shape}')
#     temp = canvas[(h-new_h)//2:(h-new_h)//2 + new_h,(w-new_w)//2:(w-new_w)//2 + new_w,  :]
#     print(temp.shape)
    canvas[(h-new_h)//2:(h-new_h)//2 + new_h,(w-new_w)//2:(w-new_w)//2 + new_w,  :] = resized_image
    
    img = canvas

    scale = min(size/img_h, size/img_w)
    bbox[:,:4] *= (scale)

    new_w = scale*img_w
    new_h = scale*img_h  

    del_h = (size - new_h)/2
    del_w = (size - new_w)/2

    add_matrix = np.array([[del_w, del_h, del_w, del_h]]).astype(int)

    bbox[:,:4] += add_matrix

    img = img.astype(np.uint8)
    
    return img, bbox